# Create Embeddings with CNN

In this notebook, we use dataset created by @ummaymariamuna and embed all the videos from that dataset using custom CNN model.

Dataset Structure:

```
/train
|- anomalous
|- nonanomalous
```



In [1]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [2]:
# Set the device to use (e.g., 'cpu', 'cuda', 'mps')
device = torch.device("mps" if torch.backends.mps.is_available() else
                      ("cuda" if torch.cuda.is_available() else "cpu"))

# Select Device According to Availability
print("Device selected:", device)

# If the device is CUDA, print the device capability
if device.type == "cuda":
    os.system("nvidia-smi")
    print()
    print("Device type:", device.type)
    print("Capability:", torch.cuda.get_device_capability(device))
else:
    print("Device capabilities are limited on MPSs and CPUs.")

Device selected: cuda
Sat Mar 16 11:46:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              25W / 250W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+---

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1).to(device)
        self.conv2 = nn.Conv2d(64, 128, 3, 1, 1).to(device)
        self.conv3 = nn.Conv2d(128, 256, 3, 1, 1).to(device)
        self.conv4 = nn.Conv2d(256, 512, 3, 1, 1).to(device)
        self.conv5 = nn.Conv2d(512, 512, 3, 1, 1).to(device)
        self.pool = nn.MaxPool2d(2, 2).to(device)
        self.fc1 = nn.Linear(512 * 7 * 7, 4096).to(device)
        self.fc2 = nn.Linear(4096, 2048).to(device)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 512 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
    
model = CNN()
model.to(device)
model.eval()

CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25088, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=2048, bias=True)
)

Now that we set our device, move on to model

In [4]:
def get_frames(path):
    video = cv2.VideoCapture(path)
    if not video.isOpened():
        raise IOError("Error reading video file")

    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    for i in range(0, frame_count, 3):
        video.set(1, i)
        ret, frame = video.read()
        if ret:
            frame = cv2.resize(frame, (224, 224))
            frame = frame.astype("float32") / 255.0
            frames.append(frame)
    video.release()
    return np.array(frames)

In [5]:
import numpy as np
import glob
from random import shuffle
import tqdm

train_path = '/kaggle/input/augmenteddataset/train/*/*.mp4'
train_paths = glob.glob(train_path)
train_labels = [1 if i.split('/')[5] == 'anomalous' else 0 for i in train_paths]

train_zip = list(zip(train_paths, train_labels))
shuffle(train_zip)
train_paths, train_labels = zip(*train_zip)

train_labels = list(train_labels)

In [6]:
def pad(x):
    assert len(x.shape) == 4
    
    max_frames = 300 // 3
    max_height = 224
    max_width = 224
    max_channel = 3
    
    padded_x = []
    
    pad_widths = [(0, max_frames - x.shape[0]),
                  (0, max_height - x.shape[1]),
                  (0, max_width - x.shape[2]),
                  (0, max_channel - x.shape[3])]
    
    padded_x = np.pad(x, pad_widths, mode='constant', constant_values=0.0)
    
    del pad_widths
    
    return padded_x
    

In [7]:
from tqdm import tqdm

video_embeddings = []
video_labels = []

# test_path = train_paths[:20]
# test_label = train_labels[:20]

for path, label in zip(tqdm(train_paths), train_labels):
    try:
        frames = get_frames(path)

        # check the shape
        if frames.shape[0] != 100 or frames.shape[1] != 224 or frames.shape[2] != 224 or frames.shape[3] !=3:
            print(f"Shape mismatch for video {path}")
            print(f"Found Shape: {frames.shape}")
            print("Fixing shape by Padding")
            frames = pad(frames)
            print(f"Fixed Shape: {frames.shape}")

        # Reshaping the frame for CNN input
        frames = torch.tensor(frames).permute(0, 3, 1, 2).to(device)

        # Get embeddings for each frame
        embeddings_list = []
        for frame in frames:
            frame = frame.unsqueeze(0)
            frame.to(device)
            embeddings = model(frame)
            embeddings_list.append(embeddings.flatten().cpu().detach().numpy())

        embeddings_list = np.asarray(embeddings_list)    
        video_embeddings.append(embeddings_list)
        video_labels.append(label)
    except IOError as e:
        print(f"Skipping Video file {path} and label {label}")
    
video_embeddings = np.asarray(video_embeddings)
video_labels = np.asarray(video_labels)

  1%|          | 103/9150 [01:11<1:45:09,  1.43it/s]

Shape mismatch for video /kaggle/input/augmenteddataset/train/nonanomalous/Normal_Videos566_x264_089.mp4
Found Shape: (98, 224, 224, 3)
Fixing shape by Padding
Fixed Shape: (100, 224, 224, 3)


  2%|▏         | 146/9150 [01:40<1:39:11,  1.51it/s]

Shape mismatch for video /kaggle/input/augmenteddataset/train/nonanomalous/Normal_Videos635_x264_003.mp4
Found Shape: (84, 224, 224, 3)
Fixing shape by Padding
Fixed Shape: (100, 224, 224, 3)


  6%|▋         | 582/9150 [06:36<1:38:16,  1.45it/s]

...

 98%|█████████▊| 8991/9150 [1:42:09<01:48,  1.47it/s]

Shape mismatch for video /kaggle/input/augmenteddataset/train/nonanomalous/Normal_Videos600_x264_006.mp4
Found Shape: (84, 224, 224, 3)
Fixing shape by Padding
Fixed Shape: (100, 224, 224, 3)


 99%|█████████▉| 9040/9150 [1:42:43<01:17,  1.42it/s]

Shape mismatch for video /kaggle/input/augmenteddataset/train/nonanomalous/Normal_Videos652_x264_010.mp4
Found Shape: (98, 224, 224, 3)
Fixing shape by Padding
Fixed Shape: (100, 224, 224, 3)


100%|██████████| 9150/9150 [1:43:58<00:00,  1.47it/s]


In [8]:
np.save('/kaggle/working/train_embeddings.npy', video_embeddings) 
np.save('/kaggle/working/train_labels.npy', video_labels)

## Checking the saved shape

In [9]:
x = np.load('/kaggle/working/train_embeddings.npy')
print(x.shape)
y = np.load('/kaggle/working/train_labels.npy')
print(y.shape)

(9150, 100, 2048)
(9150,)
